In [14]:
def points_to_take(rep,mat):
    if rep != 'Retrieval_new_rec/':
        to_take, time = [54, 99], mat['time'][54:99]-3
    else:
        to_take, time = [39, 119], mat['time'][39:119]-5
    return(to_take, time)

In [25]:
"""
SUM UP CLASSIF RESULTS for POWER
new scripts with sklearn and ALL or AVERAGE values of POWER
"""
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product
from brainpipe.statistics import perm_pvalue2level

st = study('Ripples')
conds = ['low','high']
val = 'xpow'
#reps = ['Encoding/', 'Retrieval_new_odors/', 'Retrieval_new_rec/']
reps = ['Retrieval_new_odors/']
average = False

path_classif = join(st.path, 'classified/{}')
clf_name = join(path_classif, '{}_LDA_clf_{}_{}_{}.npz')
path_pow = join(st.path, 'feature/{}')
pow_file = join(path_pow, '{}_cond={}_bipo_feat_norm.npz')
path2save = path_classif

subjects = ['FERJ','LEFC','SEMC','PIRJ','VACJ','CHAF']
freqs = ['HFA','ripple','low_freq']

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for rep,freq in product(reps,freqs):
    
    su_codes, s_xyz, s_channels,s_labels = np.array([]),np.array([]),np.array([]),np.array([])
    s_elec_pow0, s_elec_pow1, s_auc = np.array([]),np.array([]), np.array([])
    s_elec, s_perm = np.array([]),np.array([])
    for i,su in enumerate((subjects)): #sorted
        
        pow_mat = np.load(pow_file.format(rep,su,conds[0]),allow_pickle=True)
        id_freq = [i for i,f in enumerate(pow_mat['fname']) if f==freq][0]
        to_take, time = points_to_take(rep,pow_mat)
        pow0 = pow_mat['xpow'][id_freq,:,to_take[0]:to_take[1],:]
        pow1 = np.load(pow_file.format(rep,su,conds[1]))['xpow'][id_freq,:,to_take[0]:to_take[1],:]
        classif = np.load(clf_name.format(rep,su,freq,conds[0],conds[1]),allow_pickle=True)
        
        if average == True:
            pow0, pow1 = np.mean(pow0, axis=(1,2)), np.mean(pow1,axis=(1,2))
        elif average == False:
            pow0, pow1 = np.mean(pow0, axis=-1), np.mean(pow1, axis=-1)
        
        xyz, channels = classif['xyz'], classif['channels']
        auc, names, perm = classif['auc'], classif['names'], classif['perm']
        
        s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
        su_codes = np.hstack((su_codes,np.array([su]*xyz.shape[0]))) \
                                if np.size(su_codes) else np.array([su]*xyz.shape[0])
        s_elec = np.hstack((s_elec,np.arange(xyz.shape[0]))) \
                                if np.size(s_elec) else np.arange(xyz.shape[0])
        s_channels = np.hstack((s_channels, channels)) \
                                if np.size(s_channels) else channels
        s_auc = np.vstack((s_auc, auc)) if np.size(s_auc) else auc
        s_perm = np.vstack((s_perm,perm)) if np.size(s_perm) else perm
        s_labels = np.hstack((s_labels, names)) if np.size(s_labels) else names
        s_elec_pow0 = np.vstack((s_elec_pow0, pow0)) if np.size(s_elec_pow0) else pow0
        s_elec_pow1 = np.vstack((s_elec_pow1, pow1)) if np.size(s_elec_pow1) else pow1
        print(s_elec_pow0.shape, s_elec.shape, s_xyz.shape)
    print('all names', np.unique(s_labels))
    print ('shapes', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_auc.shape, 
           'perm',s_perm.shape,'su_codes',su_codes.shape, 's_elec',s_elec.shape,
          's_labels',s_labels.shape,'s_channels',s_channels.shape)
    su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                      's_elec_pow0':s_elec_pow0,'s_elec_pow1':s_elec_pow1,
                      's_da':s_auc,'s_perm':s_perm,'s_elec':s_elec, 's_channels':s_channels,
                     's_time':time}
    file_source = 'All_subjects_sources_'+freq+'_'+conds[0]+'_'+conds[1]+'.npz'
    np.savez(path2save.format(rep)+file_source,**su_coord_label)


-> Ripples loaded
(62, 45) (62,) (62, 3)
(171, 45) (171,) (171, 3)
(214, 45) (214,) (214, 3)
(279, 45) (279,) (279, 3)
(370, 45) (370,) (370, 3)
(429, 45) (429,) (429, 3)
all names ['CD' 'Frontal' 'INS' 'MTL' 'OFC_OLF' 'PHG_FuG' 'Parietal' 'aMTL'
 'cingulate' 'occipital' 'olf' 'orbital' 'temp_cx']
shapes pow1 (429, 45) pow0 (429, 45) da (429, 45) perm (429, 45, 100) su_codes (429,) s_elec (429,) s_labels (429,) s_channels (429,)
(62, 45) (62,) (62, 3)
(171, 45) (171,) (171, 3)
(214, 45) (214,) (214, 3)
(279, 45) (279,) (279, 3)
(370, 45) (370,) (370, 3)
(429, 45) (429,) (429, 3)
all names ['CD' 'Frontal' 'INS' 'MTL' 'OFC_OLF' 'PHG_FuG' 'Parietal' 'aMTL'
 'cingulate' 'occipital' 'olf' 'orbital' 'temp_cx']
shapes pow1 (429, 45) pow0 (429, 45) da (429, 45) perm (429, 45, 100) su_codes (429,) s_elec (429,) s_labels (429,) s_channels (429,)
(62, 45) (62,) (62, 3)
(171, 45) (171,) (171, 3)
(214, 45) (214,) (214, 3)
(279, 45) (279,) (279, 3)
(370, 45) (370,) (370, 3)
(429, 45) (429,) (429, 3)

#### Create a df with all information (elecs, max da, power when da is max)

In [28]:
from brainpipe.system import study
from os.path import isfile, join
import numpy as np
from itertools import product
import pandas as pd

conds = ['low','high']
rep = 'Retrieval_new_odors/' #'Retrieval_new_odors/', 'Retrieval_new_rec/', 'Encoding/'
st = study('Ripples')
path_npz = join(st.path, 'classified/'+rep)
f_form = 'All_subjects_sources_{}_{}_{}.npz'
f_form = join(path_npz, f_form)
dfname = '0_all_subjects_info_elecs.csv'
df_all_name = '1_all_subjects_AUC_Pow_Time_bsl_all_freqs.csv'
###############################################################################
freqs = ['HFA','ripple','low_freq']
data_type = 'pow'
###############################################################################

mat = np.load(f_form.format(freqs[0],conds[0],conds[1]),allow_pickle=True)
subjects = mat['su_codes'][:,np.newaxis]
elecs = mat['s_elec'][:,np.newaxis]
s_Mai_RL = mat['s_labels'][:,np.newaxis]
channels = mat['s_channels'][:,np.newaxis]
elec_data = np.concatenate((subjects, s_Mai_RL,channels,elecs), axis=1)
df0 = pd.DataFrame(elec_data, columns=['subjects','s_Mai_RL','channels','elecs_num'])
df0.to_csv(path_npz+dfname)

# load all electrodes information and create a df
idx_all, pow0_all, pow1_all, da_all = np.array([]),np.array([]),np.array([]),np.array([])
for freq in freqs:
    mat = np.load(f_form.format(freq,conds[0],conds[1]),allow_pickle=True)
    idx_elecs, pow0_elecs, pow1_elecs, da_elecs = np.array([]),np.array([]),np.array([]),np.array([])
    for elec in range(elecs.shape[0]):
        da, time = mat['s_da'][elec], mat['s_time']
        da_th = np.percentile(da,95)
        val0 = 's_elec_pow0' if data_type == 'pow' else 'rsa0'
        val1 = 's_elec_pow1' if data_type == 'pow' else 'rsa1'
        if np.size(da) > 1:
            idx_elec = np.asarray([i for i,j in enumerate(da) if j >= da_th],dtype=int)
            if len(idx_elec) < 1:
                da_th = da_th - 0.001
                idx_elec = np.asarray([i for i,j in enumerate(da) if j >= da_th],dtype=int)
            first_time = time[idx_elec][0]
            da_elec = np.mean(da[idx_elec])
            pow0_elec = np.mean(mat[val0][elec][idx_elec])
            pow1_elec = np.mean(mat[val1][elec][idx_elec])
        else:
            da_elec = da
            pow0_elec = mat[val0][elec]
            pow1_elec = mat[val1][elec]
        pow0_elecs = np.hstack((pow0_elecs,pow0_elec)) if np.size(pow0_elecs) else pow0_elec
        pow1_elecs = np.hstack((pow1_elecs, pow1_elec)) if np.size(pow1_elecs) else pow1_elec
        idx_elecs = np.hstack((idx_elecs, first_time)) if np.size(idx_elecs) else first_time
        da_elecs = np.hstack((da_elecs,da_elec)) if np.size(da_elecs) else da_elec
    idx_all = np.vstack((idx_all, idx_elecs)) if np.size(idx_all) else idx_elecs
    pow0_all= np.vstack((pow0_all, pow0_elecs)) if np.size(pow0_all) else pow0_elecs
    pow1_all= np.vstack((pow1_all, pow1_elecs)) if np.size(pow1_all) else pow1_elecs
    da_all = np.vstack((da_all, da_elecs)) if np.size(da_all) else da_elecs
df2 = pd.DataFrame(da_all.T, columns=[freq+'_AUC' for freq in freqs])
df3 = pd.DataFrame(pow0_all.T, columns=[freq+'_Pow0' for freq in freqs])
df4 = pd.DataFrame(pow1_all.T, columns=[freq+'_Pow1' for freq in freqs])
df5 = pd.DataFrame(idx_all.T, columns=[freq+'_first_T' for freq in freqs])
df_all = pd.concat((df0,df2,df3,df4,df5),axis=1)
df_all.to_csv(path_npz+df_all_name)

-> Ripples loaded
